In [ ]:
# 코랩에서 mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

stop word
URL 나오도록

- 자카드 코사인 비율
- 임계치
- 품사 태깅

In [ ]:
# 구글 다라이브 연결 코드
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# 필요시 Mecab 수정
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
import numpy as np
import time
import re


In [ ]:
df_stop = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/한국어불용어100.txt', sep='\t', header=None)
stop_token_list = []
for idx in df_stop.index:
    stop_token_list.append((df_stop.loc[idx, 0], df_stop.loc[idx, 1]))
stop_token_list

In [ ]:
# 다음 뉴스 본문
df = pd.read_csv('/content/drive/MyDrive/네이버_뉴스_추천/유사도/society_daum_news.csv')
df = df.drop_duplicates('URL')
df = df.drop_duplicates('Content')
df = df.dropna(subset=['Content'])
df = df.reset_index(drop=True)
df

,Unnamed: 0,Content,DomainID,MainCategory,PhotoURL,Press,Stickers,SubCategory,Title,URL,WritedAt,Writer
0,0,"국회 행안위 소위 25억 삭감,전자충격기, 순찰차 유류비도 줄줄이 깎여[아시아경제 ...",0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,아시아경제,"{'article_id': '20221130104657051', '좋아요': 133...",사건/사고,尹대통령 주문한 ‘1인 1총기’ 예산 25억·전자충격기 전액 삭감,https://v.daum.net/v/20221130104657051,2022. 11. 30. 10:46,장세희
1,1,(부산=연합뉴스) 손형주 기자 = 음주 측정을 거부하며 경찰관 3명을 폭행한 20대...,0,사회,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,연합뉴스,"{'article_id': '20221130105301310', '좋아요': 0, ...",사건/사고,음주 측정 거부하고 경찰관 3명 폭행한 20대 검거,https://v.daum.net/v/20221130105301310,2022. 11. 30. 10:53,손형주
2,2,이태원역장도 오늘 소환(서울=뉴스1) 손승환 기자 = 이태원 참사를 수사 중인 경찰...,0,사회,https://img1.daumcdn.net/thumb/R658x0.q70/?fna...,뉴스1,"{'article_id': '20221130105309315', '좋아요': 0, ...",사건/사고,"특수본 출석 용산구청 부청장 ""죄송하다…성실히 수사 임할 것""",https://v.daum.net/v/20221130105309315,2022. 11. 30. 10:53,손승환 기자
3,3,국민의힘과 더불어민주당은 지난 23일 이태원 참사 국정조사를 합의했다. 24일부터 ...,0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,더팩트,"{'article_id': '20221125000025485', '좋아요': 1, ...",사건/사고,"이태원 참사 국정조사, 특수본 수사에 '약'일까 '독'일까",https://v.daum.net/v/20221125000025485,2022. 11. 25. 00:00,최의종
4,4,"서산서 60대 엽사, 동료 총에 숨져,경찰, 업무상과실치사 혐의로 입건,최근 포상금...",0,사회,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,동아일보,"{'article_id': '20221114030332296', '좋아요': 2, ...",사건/사고,"또 멧돼지로 오인사격, 올해만 3번째 사망자",https://v.daum.net/v/20221114030332296,2022. 11. 14. 03:03,서산=이기진 기자
...,...,...,...,...,...,...,...,...,...,...,...,...
54027,58047,대전지검 천안지청은 28일 박상돈 천안시장에 대해 지난 6.1 지방선거당시 공무원 ...,0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,더팩트,"{'article_id': '20221129110734310', '좋아요': 0, ...",전국,천안·아산 시장 ‘공직선거법 위반’혐의로 재판 받는다,https://v.daum.net/v/20221129110734310,2022. 11. 29. 11:07,김경동
54028,58048,벡스코는 28일 부산 서구노인복지관에 지역특산품 200세트(300만 원 상당)를 기...,0,사회,https://img4.daumcdn.net/thumb/R658x0.q70/?fna...,노컷뉴스,"{'article_id': '20221129110004878', '좋아요': 0, ...",전국,"벡스코, 노인복지관에 지역특산품 200세트 기부",https://v.daum.net/v/20221129110004878,2022. 11. 29. 11:00,부산CBS 박상희 기자
54029,58049,30일(수) 오전 0시 '동파 주의(2단계)' 발령30일부터 서울 전역에 아침 최저...,0,사회,https://img3.daumcdn.net/thumb/R658x0.q70/?fna...,노컷뉴스,"{'article_id': '20221129110008889', '좋아요': 0, ...",전국,30일 최저 영하7도…서울 수도계량기 '동파주의' 발령,https://v.daum.net/v/20221129110008889,2022. 11. 29. 11:00,CBS노컷뉴스 김민수 기자
54030,58050,"내일 아침 체감온도 영하 15도…""등굣길·출근길 추위 대비"",이번 추위, 다음 달 ...",0,사회,https://img2.daumcdn.net/thumb/R658x0.q70/?fna...,노컷뉴스,"{'article_id': '20221129110309117', '좋아요': 0, ...",전국,"충남권 한파특보…오늘 밤부터 영하권 ""퇴근길 추워요""",https://v.daum.net/v/20221129110309117,2022. 11. 29. 11:03,대전CBS 김미성 기자


In [104]:
# 함수 튜닝
def preprocessing(idx):
    content = df.loc[idx, 'Content']
    # 숫자 한글 영문 , . 공백을 제외한 문자 제거
    content_pre = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣. ]', '', content)
    # 소수 및 정수 제거
    content_pre = re.sub('\d+.\d*', '',content_pre)
    # url 및 이메일 제거
    content_pre = re.sub("([a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", '', content_pre)
    # 다수의 점 (ex : ...) 점 한개로 대체
    content_pre = re.sub("\.+\.", '.', content_pre)
    # 다수의 공백 축소
    content_pre = re.sub(' +', ' ', content_pre)

    # 기자명, 언론사명 stopword 처리
    wrt = df.loc[idx, 'Writer']
    press = df.loc[idx, 'Press']

    # stopword 추가해서 처리
    stop_word = [f'{wrt}', f'{press}', '기자', 'Copyrights',  '무단 전재 및 재배포 금지']
    content_pre2 = content_pre
    for stp in stop_word :
        content_pre2 = content_pre2.replace(stp, '')
    return content_pre2
    
# 데이터프레임 토큰화
def get_df_token(data, input_col, output_col):

    mecab = Mecab()
    q_pos = []

    for i in tqdm(range(len(data))):
        q_pos.append(mecab.pos(data[input_col][i]))
    data[output_col] = q_pos

    # 품사 태깅 얼마나 사용할지 조정 필요
    pos = ["NNG","NNP",   # 명사
            "VV","VA",   # 동사
                "MA", # 부사
            ]
    # stop_word 추가
    data[output_col] = data[output_col].apply(lambda x: [t for (t, p) in x if (t,p) not in stop_token_list if p in pos])

    return data

# 질문 토큰화
def get_ques_token(content_token):
    mecab = Mecab()
    ques_token = mecab.pos(content_token)
    ques_token = ques_remove(ques_token)

    return ques_token

# 질문 토큰화 태깅 제거
def ques_remove(content_token) :

    pos = ["NNG","NNP",   # 명사
            "VV","VA",   # 동사
                "MA", # 부사
            ]
    # stop_word 추가
    ques_token = [[t for (t, p) in content_token if (t,p) not in stop_token_list if p in pos]]

    return ques_token

def jaccard_similarity(data, content_token):

    union = set(data).union(set(content_token))
    intersection = set(data).intersection(set(content_token))

    jaccard_sim = len(intersection) / len(union)  

    return jaccard_sim

def jaccard_high(data, content_token, num, input_col, output_col):
    
    # data: 데이터프레임, content_token: 입력한 텍스트(content_token), num: 자카드 유사도 상위 갯수
    data['jaccard_similarity'] = data[output_col].apply(lambda x: jaccard_similarity(x, content_token[0]))

    return data[[input_col, output_col, 'jaccard_similarity']].sort_values(['jaccard_similarity'], ascending=False)[:num]

def tokenized_output(tokens):
    return tokens

def cos_similarity(data, content_token, input_col, output_col):

    tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                       tokenizer=tokenized_output,
                                       preprocessor=tokenized_output,
                                       token_pattern=None)
    
    tfidf_data = tfidf_vectorizer.fit_transform(data[output_col])
    tfidf_content_token = tfidf_vectorizer.transform(content_token)

    data['cosine_similarity'] = cosine_similarity(tfidf_data, tfidf_content_token).reshape(-1, )

    return data[[input_col, output_col, 'jaccard_similarity', 'cosine_similarity']].sort_values(['cosine_similarity'], ascending=False)

def cos_high(data, input_col, output_col):
    if data['cosine_similarity'][0] >= 0.6:
        return data[input_col][0]

    else:
        return print('모델구현')

# 전처리 및 토큰화 함수. 토큰화 미리 시켜준값 대입시키려면 튜닝 필요. 시간도 훨씬 줄어들 예정
def top_100(df_temp, target_column):
    st_time = time.time()
    # 전처리 동작
    df_content = pd.DataFrame(columns=['content'])
    print('전처리 시작 :')
    for idx in tqdm(df_temp.index):
        temp_content = preprocessing(idx)
        temp_df = pd.DataFrame({'content' : temp_content}, index=[0])
        df_content = pd.concat([df_content, temp_df])
    df_content = df_content.reset_index(drop=True)
    
    # 토큰화. 토큰화 된거 사용하려면 함수 수정 필요.ㅣ
    mecab = Mecab()
    q_pos = []
    print('토큰화 중 : ')
    # pos 태깅 
    for i in tqdm(range(len(df_content))):
        q_pos.append(mecab.pos(df_content['content'][i]))
    df_content['content_token'] = q_pos
    # post 태깅 검증
    print('pos_태깅_검증')
    df_content = get_df_token(df_content, 'content', 'content_token')
    end_time = time.time()
    print('토큰화 소요시간 :', end_time - st_time, '\n')

    # df_content 여기에서 저장 
##################################################################################
#### 위까지 토큰화 코드, 아래는 토큰화 된 컬럼 df_content 기준으로 함수 변경하면 됨
#### 다만, 컬럼명 확인이 반드시 필요 #############################################
##################################################################################
    st_time = time.time()

    # 타겟 본문 토큰
    ques_token = get_ques_token(target_column['Content'])

    result_temp = jaccard_high(df_content, ques_token, 100, 'content', 'content_token') # 전체데이터프레임길이 
    result_temp = cos_similarity(result_temp, ques_token, 'content', 'content_token')
    end_time = time.time()
    print('계산 소요시간 :', end_time - st_time, '\n')
    return result_temp

def total_score_cal(df_temp, result_temp, target_column) :
    st_time = time.time()
    # 뉴스 인덱스 처리
    result_temp2 = result_temp.copy()
    result_temp2 = result_temp2.reset_index()
    result_temp2 = result_temp2.rename(columns={'index' : 'url_index'})
    result_temp2 = result_temp2.reset_index()

    # 동일기사 검증 임계치
    threshold = 0.25
    result_temp2 = result_temp2[(result_temp2['jaccard_similarity'] < threshold) & (result_temp2['cosine_similarity'] < (threshold + 0.05))]

    # 필요한 컬럼 가져오기
    for col in ['MainCategory', 'SubCategory', 'WritedAt', 'URL']:
        for idx in result_temp2.index :
            result_temp2.loc[idx, col] = df_temp.loc[result_temp2.loc[idx, 'url_index'], col]

    # 타겟 타이틀 불러오기
    # target_column = df.iloc[0, :]
    target_title = target_column['Title']

    # 타이틀 토큰된거로 유사도 계산
    ques_token_title = get_ques_token(target_title)
    ques_token_title

    result_temp3 = result_temp2.copy()
    result_temp3 = jaccard_high(result_temp3, ques_token_title, 100, 'content', 'content_token')
    result_temp3 = cos_similarity(result_temp3, ques_token_title, 'content', 'content_token')
    result_temp3 = result_temp3.rename(columns={'jaccard_similarity' : 'Ti_jaccard_similarity', 'cosine_similarity' : 'Ti_cosine_similarity'})
    result_temp3 = result_temp3.reset_index()

    # 타이틀 토큰화 결과 합치기
    result_temp2 = pd.merge(result_temp2, result_temp3[['index', 'Ti_jaccard_similarity', 'Ti_cosine_similarity']], how='left', on='index')
    del result_temp2['index']

    # total_score
    # 유사도 스캐일링
    content_ja_scaled = (result_temp2['jaccard_similarity'] - result_temp2['jaccard_similarity'].min()) /( result_temp2['jaccard_similarity'].max() - result_temp2['jaccard_similarity'].min())
    content_co_scaled = (result_temp2['cosine_similarity'] - result_temp2['cosine_similarity'].min())/( result_temp2['cosine_similarity'].max() - result_temp2['cosine_similarity'].min())
    cotent_score = (2/5 * 50) * content_ja_scaled + (3/5 * 50) * content_co_scaled

    # 본문 핵심문장
    tr_content_ja_scaled = (result_temp2['Ti_jaccard_similarity'] -  result_temp2['Ti_jaccard_similarity'].min() )/ (result_temp2['Ti_jaccard_similarity'].max() -  result_temp2['Ti_jaccard_similarity'].min())
    tr_content_co_scaled = (result_temp2['Ti_cosine_similarity'] -  result_temp2['Ti_cosine_similarity'].min())/ (result_temp2['Ti_cosine_similarity'].max() -  result_temp2['Ti_cosine_similarity'].min())
    tr_cotent_score = (2/5 * 30) * tr_content_ja_scaled + (3/5 * 30) * tr_content_co_scaled

    # 카테고리
    main_cate_score = np.where(result_temp2['MainCategory'] == target_column['MainCategory'], 1, 0) * 0.1
    sub_cate_score = np.where(result_temp2['SubCategory'] == target_column['SubCategory'], 1, 0) * 0.05

    # 날짜
    result_date = pd.to_datetime(result_temp2['WritedAt']).dt.day.values
    target_date = int(pd.to_datetime(target_column['WritedAt']).day)
    date_score = np.where(abs(result_date - target_date) <= 1, 1, 
                        np.where(abs(result_date - target_date) <= 3, 0.5, 
                                    np.where(abs(result_date - target_date) <= 7, 0.3, 0))) * 0.05

    total_score = cotent_score + tr_cotent_score + main_cate_score + sub_cate_score + date_score
    result_temp2['total_score'] = total_score
    result_temp2 = result_temp2.sort_values(by='total_score', ascending=False)
    result_temp2['total_score'] = result_temp2['total_score'].apply(lambda x : round(x,2))
    end_time = time.time()
    print('소요시간 :', end_time - st_time, '\n')
    return result_temp2

In [105]:
df_top_100 = top_100(df, df.iloc[54027, :])
df_result = total_score_cal(df, df_top_100, df.iloc[54027, :])
df_result

전처리 시작 :


  0%|          | 0/54032 [00:00<?, ?it/s]

토큰화 중 : 


  0%|          | 0/54032 [00:00<?, ?it/s]

pos_태깅_검증


  0%|          | 0/54032 [00:00<?, ?it/s]

토큰화 소요시간 : 322.7861886024475 

계산 소요시간 : 1.370147705078125 

소요시간 : 0.11915135383605957 



,url_index,content,content_token,jaccard_similarity,cosine_similarity,MainCategory,SubCategory,WritedAt,URL,Ti_jaccard_similarity,Ti_cosine_similarity,total_score
2,22911,정읍 허위사실을 공표한 혐의를 받는 이학수 전북 정읍시장이 재판에 넘겨졌다.전주지검...,"[정읍, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 재판, 전주, 지검, ...",0.234043,0.217194,사회,전라,2022. 11. 30. 16:31,https://v.daum.net/v/20221130163151711,0.136364,0.250338,67.26
0,5620,검찰이 허위사실 공표 혐의를 받아온 이학수 전북 정읍시장을 기소했다.전주지검 정읍지...,"[검찰, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 기소, 전주, 지검, ...",0.223404,0.238536,사회,사건/사고,2022. 11. 30. 17:40,https://v.daum.net/v/20221130174005281,0.113636,0.200339,64.79
8,14353,정읍 지난 방선거에서 허위사실를 공표한 혐의를 받는 이학수 전북 정읍시장이 재판...,"[정읍, 방, 선거, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 재판, 전...",0.239130,0.192892,사회,지역일반,2022. 11. 30. 16:48,https://v.daum.net/v/20221130164835424,0.142857,0.218007,61.72
1,35942,대전충남뉴스 대전지검 형사가 지난 일 지방선거 당시 확성장치를 이용해 선거운동을 한...,"[대전, 충남, 뉴스, 대전, 지검, 형사, 지방, 선거, 당시, 확, 장치, 이용...",0.189873,0.219819,사회,충청,2022. 11. 22. 16:28,https://v.daum.net/v/20221122162844237,0.217391,0.152521,56.41
6,53116,앞서 경찰은 박 시장을 공직선거법 위반 혐의로 검찰에 송치했다. 박 시장은 최근 구...,"[경찰, 박, 시장, 공직, 선거법, 위반, 혐의, 검찰, 송치, 박, 시장, 최근...",0.197674,0.194951,사회,전국,2022. 11. 18. 14:16,https://v.daum.net/v/20221118141642416,0.156250,0.233192,55.58
...,...,...,...,...,...,...,...,...,...,...,...,...
73,45089,인천뉴스 지난 지방선거 당시 상대측 후보에 대한 허위사실 공표와 특정정당의 지지를 ...,"[인천, 뉴스, 지방, 선거, 당시, 상대측, 후보, 허위, 공표, 특정, 정당, ...",0.160377,0.102877,사회,수도권,2022. 11. 25. 17:47,https://v.daum.net/v/20221125174713037,0.036364,0.024361,10.33
84,675,경찰에 따르면 서울경찰청 반부패공공범죄수사대는 전날 공직선거법 위반 혐의를 받는 ...,"[경찰, 서울, 경찰청, 반부, 패공, 공범죄, 수사대, 전날, 공직, 선거법, 위...",0.173228,0.069199,사회,사건/사고,2022. 11. 24. 12:40,https://v.daum.net/v/20221124124013564,0.050633,0.043620,8.37
85,7131,남양주 검찰이 선거를 앞두고 이해관계인의 골프장 라운딩 예약 부탁을 들어준 서태원 ...,"[남양주, 검찰, 선거, 앞두, 이해관계인, 골프장, 라운, 딩, 예약, 부탁, 서...",0.165049,0.066606,사회,지역일반,2022. 11. 29. 17:31,https://v.daum.net/v/20221129173149282,0.080000,0.047674,8.01
77,49340,김 군수 등은 지난 후보자 TV토론 때부터 후보자 매수 의혹에 휩싸이기 시작했다. ...,"[김, 군수, 후보자, 토론, 후보, 매수, 의혹, 휩싸이, 시작, 이후, 지역, ...",0.160714,0.095141,사회,전국,2022. 11. 4. 17:20,https://v.daum.net/v/20221104172053023,0.015873,0.008314,6.84


In [101]:
df_result[(df_result['jaccard_similarity'] < 0.25) & (df_result['cosine_similarity'] < 0.3)]

,url_index,content,content_token,jaccard_similarity,cosine_similarity,MainCategory,SubCategory,WritedAt,URL,Ti_jaccard_similarity,Ti_cosine_similarity,total_score
6,22911,정읍 허위사실을 공표한 혐의를 받는 이학수 전북 정읍시장이 재판에 넘겨졌다.전주지검...,"[정읍, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 재판, 전주, 지검, ...",0.234043,0.217194,사회,전라,2022. 11. 30. 16:31,https://v.daum.net/v/20221130163151711,0.136364,0.116009,31.68
2,5620,검찰이 허위사실 공표 혐의를 받아온 이학수 전북 정읍시장을 기소했다.전주지검 정읍지...,"[검찰, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 기소, 전주, 지검, ...",0.223404,0.238536,사회,사건/사고,2022. 11. 30. 17:40,https://v.daum.net/v/20221130174005281,0.113636,0.093930,30.22
12,14353,정읍 지난 방선거에서 허위사실를 공표한 혐의를 받는 이학수 전북 정읍시장이 재판...,"[정읍, 방, 선거, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 재판, 전...",0.239130,0.192892,사회,지역일반,2022. 11. 30. 16:48,https://v.daum.net/v/20221130164835424,0.142857,0.101301,30.01
5,35942,대전충남뉴스 대전지검 형사가 지난 일 지방선거 당시 확성장치를 이용해 선거운동을 한...,"[대전, 충남, 뉴스, 대전, 지검, 형사, 지방, 선거, 당시, 확, 장치, 이용...",0.189873,0.219819,사회,충청,2022. 11. 22. 16:28,https://v.daum.net/v/20221122162844237,0.217391,0.072184,29.20
10,53116,앞서 경찰은 박 시장을 공직선거법 위반 혐의로 검찰에 송치했다. 박 시장은 최근 구...,"[경찰, 박, 시장, 공직, 선거법, 위반, 혐의, 검찰, 송치, 박, 시장, 최근...",0.197674,0.194951,사회,전국,2022. 11. 18. 14:16,https://v.daum.net/v/20221118141642416,0.156250,0.106758,26.43
...,...,...,...,...,...,...,...,...,...,...,...,...
89,7131,남양주 검찰이 선거를 앞두고 이해관계인의 골프장 라운딩 예약 부탁을 들어준 서태원 ...,"[남양주, 검찰, 선거, 앞두, 이해관계인, 골프장, 라운, 딩, 예약, 부탁, 서...",0.165049,0.066606,사회,지역일반,2022. 11. 29. 17:31,https://v.daum.net/v/20221129173149282,0.080000,0.021927,5.19
77,45089,인천뉴스 지난 지방선거 당시 상대측 후보에 대한 허위사실 공표와 특정정당의 지지를 ...,"[인천, 뉴스, 지방, 선거, 당시, 상대측, 후보, 허위, 공표, 특정, 정당, ...",0.160377,0.102877,사회,수도권,2022. 11. 25. 17:47,https://v.daum.net/v/20221125174713037,0.036364,0.011236,4.99
88,675,경찰에 따르면 서울경찰청 반부패공공범죄수사대는 전날 공직선거법 위반 혐의를 받는 ...,"[경찰, 서울, 경찰청, 반부, 패공, 공범죄, 수사대, 전날, 공직, 선거법, 위...",0.173228,0.069199,사회,사건/사고,2022. 11. 24. 12:40,https://v.daum.net/v/20221124124013564,0.050633,0.020061,4.75
90,1888,경찰에 따르면 서울경찰청 사이버범죄수사대는 송 전 대표를 지난 서울중앙지검에 불구...,"[경찰, 서울, 경찰청, 사이버, 범죄, 수사대, 송, 대표, 서울, 중앙, 지검,...",0.155172,0.066229,사회,사건/사고,2022. 11. 23. 20:20,https://v.daum.net/v/20221123202023460,0.062500,0.030668,3.50


In [99]:
# 타겟 컬럼 시리즈 형태로.
df.iloc[54027, :]

Unnamed: 0                                                  58047
Content         대전지검 천안지청은 28일 박상돈 천안시장에 대해 지난 6.1 지방선거당시 공무원 ...
DomainID                                                        0
MainCategory                                                   사회
PhotoURL        https://img4.daumcdn.net/thumb/R658x0.q70/?fna...
Press                                                         더팩트
Stickers        {'article_id': '20221129110734310', '좋아요': 0, ...
SubCategory                                                    전국
Title                               천안·아산 시장 ‘공직선거법 위반’혐의로 재판 받는다
URL                        https://v.daum.net/v/20221129110734310
WritedAt                                      2022. 11. 29. 11:07
Writer                                                        김경동
Name: 54027, dtype: object

In [103]:
# toal_score 가중치만 변경시 사용
df_result = total_score_cal(df, df_top_100, df.iloc[0, :])
df_result

소요시간 : 0.16516804695129395 



,url_index,content,content_token,jaccard_similarity,cosine_similarity,MainCategory,SubCategory,WritedAt,URL,Ti_jaccard_similarity,Ti_cosine_similarity,total_score
0,5620,검찰이 허위사실 공표 혐의를 받아온 이학수 전북 정읍시장을 기소했다.전주지검 정읍지...,"[검찰, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 기소, 전주, 지검, ...",0.223404,0.238536,사회,사건/사고,2022. 11. 30. 17:40,https://v.daum.net/v/20221130174005281,0.0,0.0,NaN
1,35942,대전충남뉴스 대전지검 형사가 지난 일 지방선거 당시 확성장치를 이용해 선거운동을 한...,"[대전, 충남, 뉴스, 대전, 지검, 형사, 지방, 선거, 당시, 확, 장치, 이용...",0.189873,0.219819,사회,충청,2022. 11. 22. 16:28,https://v.daum.net/v/20221122162844237,0.0,0.0,NaN
2,22911,정읍 허위사실을 공표한 혐의를 받는 이학수 전북 정읍시장이 재판에 넘겨졌다.전주지검...,"[정읍, 허위, 공표, 혐의, 이학수, 전북, 정읍, 시장, 재판, 전주, 지검, ...",0.234043,0.217194,사회,전라,2022. 11. 30. 16:31,https://v.daum.net/v/20221130163151711,0.0,0.0,NaN
3,35889,대전지검 형사는 이장우 대전시장을 공직선거법 위반 혐의로 불구속 기소했다고 밝혔다....,"[대전, 지검, 형사, 이장우, 대전, 시장, 공직, 선거법, 위반, 혐의, 구속,...",0.178947,0.210387,사회,충청,2022. 11. 22. 19:04,https://v.daum.net/v/20221122190457383,0.0,0.0,NaN
4,23098,광주뉴스 검찰이 공직선거법 위반 혐의로 박홍률 목포시장을 기소한 데 이어 박 시장의...,"[광주, 뉴스, 검찰, 공직, 선거법, 위반, 혐의, 박홍률, 목포시장, 기소, 이...",0.186275,0.206684,사회,전라,2022. 11. 29. 19:00,https://v.daum.net/v/20221129190009790,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
82,34680,대전충남뉴스 김광신 대전 중구청장이 공직선거법 위반 혐의로 재판을 받게 됐다. 대전...,"[대전, 충남, 뉴스, 김광신, 대전, 구청장, 공직, 선거법, 위반, 혐의, 재판...",0.183824,0.086459,사회,충청,2022. 11. 30. 18:18,https://v.daum.net/v/20221130181834564,0.0,0.0,NaN
83,44629,가평뉴스 국민의힘 당원들을 상대로 접대 의혹을 받는 서태원 경기 가평군수가 재판에 ...,"[가평, 뉴스, 국민, 힘, 당원, 상대, 접대, 의혹, 서태원, 경기, 가평, 군...",0.174603,0.073477,사회,수도권,2022. 11. 29. 16:28,https://v.daum.net/v/20221129162816926,0.0,0.0,NaN
84,675,경찰에 따르면 서울경찰청 반부패공공범죄수사대는 전날 공직선거법 위반 혐의를 받는 ...,"[경찰, 서울, 경찰청, 반부, 패공, 공범죄, 수사대, 전날, 공직, 선거법, 위...",0.173228,0.069199,사회,사건/사고,2022. 11. 24. 12:40,https://v.daum.net/v/20221124124013564,0.0,0.0,NaN
85,7131,남양주 검찰이 선거를 앞두고 이해관계인의 골프장 라운딩 예약 부탁을 들어준 서태원 ...,"[남양주, 검찰, 선거, 앞두, 이해관계인, 골프장, 라운, 딩, 예약, 부탁, 서...",0.165049,0.066606,사회,지역일반,2022. 11. 29. 17:31,https://v.daum.net/v/20221129173149282,0.0,0.0,NaN
